This is a .Net interactive notebook.
Requires .Net SDK 6.0 + [VSCode extension](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode)
------------------------------------------

It's been a long term goal to get Universal Dashboard running on IIS inside a docker image.  I finally had some free time..

So some notes on how make this docker image from scratch since it might be a while before I can come back to this!
Specifically for the UniversalDashboard + API project, most of this is now handled automatically inside the dockerfile and the finalsteps.ps1 but wanted to record learning steps along the way :)

First navigate to a new dir on local machine then pull the base Microsoft server core IIS image

In [ ]:
docker pull mcr.microsoft.com/windows/servercore/iis

then launch a container from the image in interactive mode:

In [ ]:
docker run -it mcr.microsoft.com/windows/servercore/iis

Make any desired manual changes inside the container.
Next exit the container but leave it running by pressing: CTRL + P + Q

When you have exited from the running container, you need to grab the container ID.

In [ ]:

docker ps

Next used the derived container ID to stop the running container

In [ ]:
docker stop <containerID>


Come up with a meaningful new image name, then commit the changes that you made inside the container back to your local docker repository inside a new image
For this project I have used iis-ud-and-api

In [ ]:
docker commit [CONTAINER_ID] iis-ud-and-api


The new image is now saved with the changes made inside the container.
In a default Windows installation, docker image layers are stored in C:\ProgramData\docker and split across the "image" and "windowsfilter" directories.

Right now, this image only has changes that were made interactively inside the container, but it's likely that you will also want to include files from outside to be copied into the container.

Do this using the DOCKERFILE in the root of the directory.
See [dockerfile reference](https://docs.docker.com/engine/reference/builder/) for available commands.

Quick explanation of some DOCKERFILE commands:
----------------------------------------------
FROM mcr.microsoft.com/windows/servercore/iis  <-- specifies a base image that we have based this project on
WORKDIR <path>  <-- this sets the path inside the container
COPY <path_on_host> <container_path> <-- this copies from the host machine into the container during build
SHELL ["powershell", "-command"] <-- by setting SHELL, subsequent commands in the DOCKERFILE will execute as PowerShell
RUN Install-WindowsFeature -name Web-WebSockets  <-- RUN kicks off a PowerShell command because we set SHELL above
etc etc...

Once you have the file structure in place in your project root dir on the host machine and you have the dockerfile commands ready, make sure you are in the root of the project dir and then run:

In [ ]:
docker build -t iis-ud-and-api .


docker build reads the instructions in the DOCKERFILE to create a new image with the name specified
When the build completes, you can launch a container instance based on the underlying the image by running the following:

--entrypoint powershell <-- since based on base image (mcr.microsoft.com/windows/servercore/iis) where the entrypoint is ServiceMontior.exe, we need this to be able to run PS commands interactively.
-it <-- container runs in interactive mode.  Replace this with -d when set up is completely finished so container runs detached in background.
--dns 1.1.1.1 <-- this allows the container to connect to the Internet (can be removed when no longer needed)
-p 8000:80 <-- exposes port 80 from the image as port 8000 on local host for the dashboard
-p 9000:8080 <-- exposes port 8080 from the image as port 9000 on local host for the UD API
--name <a_new_random_name> <-- container must be assigned a unique name

In [ ]:
$containerName = "daily_docker" #this has to be unique for each new container instance otherwise will error!
docker run --entrypoint powershell -it --dns 1.1.1.1 -p 8000:80 -p 9000:8080 --name $containerName iis-ud-and-api


When everything is working and ready to push to docker hub, you need to tag the image locally to make a connection between the local repo and dockerhub
In this case my dockerhub userID is: bayranshade and the repository on docker hub is simply called universaldashboard

In [ ]:
docker tag iis-ud-and-api bayranshade/universaldashboard:latest

To publish the image push it up to docker hub

In [ ]:

docker push bayranshade/universaldashboard:latest

Now the image you created with all the preconfigured changes is available for anyone to pull to their machine with the following command:

In [ ]:
docker pull bayranshade/universaldashboard:latest

Once the image is running change the dashboard tile by sending diffent types of car as a post..
While autorefresh is not configured a refresh of the browser will then show the change

Adjust the PSCustom object below so the car is (a highly aspirational!) BMW, Ferarri, or Mercedes to see the tile change.

In [ ]:
$body = [PSCustomObject]@{Car = "BMW" } | ConvertTo-Json
Invoke-RestMethod -Uri 'http://localhost:9000/api/car' -Method Post -Body $body

Example screen grabs after an update to the API

- [BMW](images/BMW.png)
- [Ferarri](images/Ferarri.png)

Some useful links that helped during this project:

- https://docs.universaldashboard.io/webserver/running-dashboards/iis
- https://stefanstranger.github.io/2018/10/02/RunningUniversalDashboardinALinuxDockerContainer/
- https://stefanstranger.github.io/2018/10/03/RunningUniversalDashboardContainerInwebAppForContainers/index.html#disqus_thread